In [1]:
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding

from ast import literal_eval as make_tuple
import ast

In [2]:
def read_from_file(train_file):
    TRAIN_DATA = []
    with open(train_file) as f:
        lines = f.read().splitlines()
    for line in lines:
        tup = make_tuple(line)
        text = tup[0].strip()
        entities = ast.literal_eval( str(tup[1]).strip())
        new_tup = (text, entities)
        TRAIN_DATA.append(new_tup)
    return TRAIN_DATA

In [3]:
TRAIN_DATA = \
[
    ('Who is Shaka Singh?', { 'entities': [(7, 17, 'PERSON')] }),
    ('I like London and Berlin.', { 'entities': [(7, 13, 'LOC'), (18, 24, 'LOC')] })
]

train_file = '/Users/sheagraw/Documents/dev/team-oracle/ner-stanford/src/main/resources/data/train/sample_dataturks.spacy'
TRAIN_DATA = read_from_file(train_file)

In [8]:
def main(model=None, output_dir=None, n_iter=100):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        # nlp = 'Language' object
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")
        
    print ("Type of the Language model '%s'" % type(nlp))
    
    # return all 'Pipe' objects in the nlp Language model
    pipes = nlp.pipeline
    print ("Pipe in the nlp model '%s'" % pipes)
    
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    # 'tokenizer', 'tagger', 'parser', 'ner', 'textcat' -- available built-ins Pipeline
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe('ner')
    
    pipes = nlp.pipeline
    print ("Pipe in the nlp model after adding ner '%s'" % pipes)
    print ("Type of the ner model '%s'" % type(ner))
    print("\n....................\n")
    
    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print('Losses', losses)
    
    # test the trained model
    for text, _ in TRAIN_DATA:
        doc = nlp(text)
        print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
        print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])
        
    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)
    
    # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in TRAIN_DATA:
            doc = nlp2(text)
            print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
            print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])
            

In [ ]:
if __name__ == '__main__':
    output_dir = '/Users/sheagraw/Documents/dev/team-oracle/ner-spacy/spacy_models/'
    main(output_dir=output_dir)